In [1]:
%pip install mediapipe
%pip install opencv-python
%pip install tensorflow
%pip install pandas 
%pip install scikit-learn
%pip install --upgrade pafy
%pip install --upgrade youtube-dl
%pip install --upgrade pytube
%pip install regex

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
from mediapipe.framework.formats import landmark_pb2
import csv
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
import regex as re

In [3]:
dir = 'Downloaded_videos'
videos = os.listdir(dir)
categories = len(videos)

In [4]:
import csv

with open('names.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['names'])  

columns = [
    'WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP',
    'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP',
    'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_TIP',
    'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_DIP', 'RING_FINGER_TIP',
    'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP'
]

new_columns = []
for col in columns:
    new_columns.append(col + '_X')
    new_columns.append(col + '_Y')

new_columns = ['LABEL']+new_columns

with open('test.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(new_columns)  

In [6]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
i=0
landmarker = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
for video in videos:
    has_hands=False
    cap = cv2.VideoCapture(f'{dir}/{video}')
    match = re.match(r'[\w\s,]+', video)
    if match:
        vid = match.group(0)
    if not cap.isOpened():
        print("Error opening video file")
        exit()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results = landmarker.process(rgb_frame)

        annotated_image = frame.copy()
        if results.multi_hand_world_landmarks:
            has_hands=True
            landmarks = [[landmark.x, landmark.y] for landmark in results.multi_hand_world_landmarks[0].landmark]
            with open('test.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([i]+[item for sublist in landmarks for item in sublist]
            )

    i+=1
    if has_hands:
        with open('names.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([vid]) 
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1725017644.675223 8041440 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1725017644.688456 8044732 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725017644.695317 8044732 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/ezk/Desktop/repositories/ytmp/.venv/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 

In [123]:


sequence_length = 25

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(sequence_length, 42)),  # 25 timesteps, 42 features
    tf.keras.layers.LSTM(200, return_sequences=True, activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.LSTM(150, return_sequences=True, activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.LSTM(100, activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(categories, activation='softmax',
                          kernel_regularizer=tf.keras.regularizers.l2(0.01))  # Output layer
])

# Print model summary
model.summary()  # Use this to visualize the model's architecture

# Model Checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    'test.keras', verbose=1, save_weights_only=False)

# Early Stopping callback
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_33 (LSTM)                  │ (None, 25, 200)        │       194,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 25, 200)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_34 (LSTM)                  │ (None, 25, 150)        │       210,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 25, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_35 (LSTM)                  │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 128)            │        12,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 518,328 (1.98 MB)

 Trainable params: 518,328 (1.98 MB)

 Non-trainable params: 0 (0.00 B)

In [124]:
df = pd.read_csv('test.csv')
names = pd.read_csv('names.csv', header=None)

In [125]:

# Assuming 'df' is already defined
# Extract features and labels
X = df.iloc[:, 1:].values  # Features (all columns except the first one)
y = df.iloc[:, 0].values   # Labels (the first column)

# Parameters
features = X.shape[1]  

num_samples = X.shape[0] // sequence_length
X = X[:num_samples * sequence_length]
y = y[:num_samples]
X_reshaped = X.reshape((num_samples, sequence_length, features))
reshaped_y = y.reshape((num_samples,))

# Print shapes to verify
print("Original X shape:", df.iloc[:, 1:].shape)
print("Reshaped X shape:", X_reshaped.shape)
print("Original y shape:", df.iloc[:, 0].shape)
print("Reshaped y shape:", reshaped_y.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, reshaped_y, test_size=0.2, random_state=42)

# Print the shapes of the splits to verify
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


Original X shape: (8189, 42)
Reshaped X shape: (327, 25, 42)
Original y shape: (8189,)
Reshaped y shape: (327,)
X_train shape: (261, 25, 42)
X_test shape: (66, 25, 42)
y_train shape: (261,)
y_test shape: (66,)


In [126]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback]
)

Epoch 1/1000
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.0293 - loss: 11.9091  
Epoch 1: saving model to test.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - accuracy: 0.0453 - loss: 11.8790 - val_accuracy: 0.0909 - val_loss: 11.3275
Epoch 2/1000
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.1328 - loss: 11.2768
Epoch 2: saving model to test.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.1315 - loss: 11.2485 - val_accuracy: 0.0909 - val_loss: 10.7314
Epoch 3/1000
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.1387 - loss: 10.6837
Epoch 3: saving model to test.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1345 - loss: 10.6572 - val_accuracy: 0.0909 - val_loss: 10.1716
Epoch 4/1000
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.1094 - loss: 10.1272
Epoch 4: saving model to test.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1198 - loss: 10.1016 - val_accuracy: 0.0909 - val_loss: 9.6368
Epoch 5/1000
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/

In [ ]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_21 (LSTM)                  │ (None, 40, 400)        │       708,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 40, 400)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 40, 200)        │       480,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 40, 200)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ (None, 100)            │       120,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 128)            │        12,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,968,786 (15.14 MB)

 Trainable params: 1,322,928 (5.05 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,645,858 (10.09 MB)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

landmarker = mp_hands.Hands(
    static_image_mode=False, 
    max_num_hands=2, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)

confidence_threshold = 0.1
frame_buffer_size = 25  # Number of frames to accumulate

# Initialize video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error opening webcam")
    exit()

frame_buffer = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = landmarker.process(rgb_frame)

    # Initialize a consistent landmark list for each frame
    landmarks = []

    if results.multi_hand_world_landmarks:
        for hand_landmarks in results.multi_hand_world_landmarks:
            # Collect all x, y, z coordinates for landmarks
            hand_landmark = [coord for landmark in hand_landmarks.landmark for coord in (landmark.x, landmark.y)]
            landmarks.append(hand_landmark)

    # Check if landmarks were found and are consistent
    if len(landmarks) > 0:
        # Flatten the landmarks list into a single vector
        flattened_landmarks = [item for sublist in landmarks for item in sublist]

        # Ensure the flattened landmarks list has the correct length (consistent for all frames)
        if len(flattened_landmarks) == 21*2:  # Assuming 21 landmarks with x, y, each
            frame_buffer.append(flattened_landmarks)
        annotated_image = frame.copy()
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # Process accumulated frames if buffer is full
        if len(frame_buffer) == frame_buffer_size:
            X = np.array(frame_buffer)  # Convert list of arrays into a numpy array
            
            # Reshape to (1, frame_buffer_size, features) for the model
            X = X.reshape((1, frame_buffer_size, -1))  # Adjust the -1 based on the number of features
            
            # Predict using the model
            y = model.predict(X)
            predicted_index = np.argmax(y)
            confidence = y[0][predicted_index]
            prediction = (names.values[predicted_index][0])
                        
            if confidence >= confidence_threshold:
                text = f"pred : ({prediction}) confidence :({confidence:.2f}) pred_index : ({predicted_index})"
                cv2.putText(annotated_image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                print(text)

            # Clear buffer after processing
            frame_buffer = []
    cv2.imshow('Live Hand Tracking', annotated_image)

    # Show the result
    
    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1724933784.020032 6303701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1724933784.054549 6520629 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724933784.061210 6520635 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
pred : (visa) confidence :(0.37) pred_index : (2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
pred : (maybe) confidence :(0.33) pred_index : (0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
pred : (visa) confidence :(0.32) pred_index : (2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
pred : (visa) confidence :(0.41) pred_index : (2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
pred : (visa) confidence :(0.49) pred_index : (2)
